In [143]:
import random as r
import pandas as pd
import numpy as np


n_samples = 10000


chars = '1234567890abcdefghijk'
user_id_len = 15
user_ids = set()
while len(user_ids) < n_samples:
    s = ''.join(r.choices(chars, k=user_id_len))
    user_ids.add(s)


max_order_num = 9
order_nums = []
for _ in range(1, n_samples + 1):
    order_nums.append(r.randint(1, max_order_num))


mean = 1440
std = 200
delivery_times = np.round(np.random.normal(mean, std, n_samples))


lambda_ = 1
shift = 1
total_sums = np.random.exponential(lambda_, n_samples) + shift 


retention_probabilities = [0.35, 0.25, 0.2, 0.15, 0.05]
retention_numbers = [1, 2, 3, 4, 5]
retentions = np.random.choice(retention_numbers, n_samples, p=retention_probabilities)


data = pd.DataFrame({
    'user_id': list(user_ids),
    'order_num': order_nums,
    'delivery_time': delivery_times,
    'total_sum': total_sums,
    'retention': retentions
    })

data

,user_id,order_num,delivery_time,total_sum,retention
0,f5fe0hi6ca1k7i4,1,1440.0,1.185157,4
1,ij7gi535ef78ea4,4,1687.0,3.483513,2
2,804c2703k8f79b0,4,1271.0,1.662753,1
3,i6k84g143cjcac0,9,1486.0,1.640576,2
4,a0d269h6hj484c9,9,1594.0,1.501818,4
...,...,...,...,...,...
9995,i3c6h8077i0g800,5,1258.0,1.270957,2
9996,3aeg17hiff8509c,9,987.0,1.637611,1
9997,38192ee2bdfg4dd,7,1261.0,1.201830,2
9998,060ei4gik3i9ja3,8,1301.0,1.146904,3


In [144]:
mean_delivery_time_data = pd.DataFrame({'mean_delivery_time': np.round(data.groupby('order_num')['delivery_time'].mean())})
merged_data = pd.merge(data, mean_delivery_time_data, on='order_num')
merged_data

,user_id,order_num,delivery_time,total_sum,retention,mean_delivery_time
0,f5fe0hi6ca1k7i4,1,1440.0,1.185157,4,1444.0
1,ij7gi535ef78ea4,4,1687.0,3.483513,2,1445.0
2,804c2703k8f79b0,4,1271.0,1.662753,1,1445.0
3,i6k84g143cjcac0,9,1486.0,1.640576,2,1439.0
4,a0d269h6hj484c9,9,1594.0,1.501818,4,1439.0
...,...,...,...,...,...,...
9995,i3c6h8077i0g800,5,1258.0,1.270957,2,1443.0
9996,3aeg17hiff8509c,9,987.0,1.637611,1,1439.0
9997,38192ee2bdfg4dd,7,1261.0,1.201830,2,1430.0
9998,060ei4gik3i9ja3,8,1301.0,1.146904,3,1443.0


In [145]:
n_in_every_segment = 3
num_segments = (max_order_num + n_in_every_segment - 1) // n_in_every_segment
bins = [i * n_in_every_segment for i in range(num_segments + 1)]
labels = [f'{i * n_in_every_segment + 1}-{(i + 1) * n_in_every_segment}' for i in range(num_segments)]

merged_data['segment'] = pd.cut(merged_data['order_num'], bins=bins, labels=labels)
merged_data

,user_id,order_num,delivery_time,total_sum,retention,mean_delivery_time,segment
0,f5fe0hi6ca1k7i4,1,1440.0,1.185157,4,1444.0,1-3
1,ij7gi535ef78ea4,4,1687.0,3.483513,2,1445.0,4-6
2,804c2703k8f79b0,4,1271.0,1.662753,1,1445.0,4-6
3,i6k84g143cjcac0,9,1486.0,1.640576,2,1439.0,7-9
4,a0d269h6hj484c9,9,1594.0,1.501818,4,1439.0,7-9
...,...,...,...,...,...,...,...
9995,i3c6h8077i0g800,5,1258.0,1.270957,2,1443.0,4-6
9996,3aeg17hiff8509c,9,987.0,1.637611,1,1439.0,7-9
9997,38192ee2bdfg4dd,7,1261.0,1.201830,2,1430.0,7-9
9998,060ei4gik3i9ja3,8,1301.0,1.146904,3,1443.0,7-9
